In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def load_flower_data():
    # grab the list of images that we'll be describing
    print("[INFO] handling images...")
    TRAIN_ORIGINAL_DIR = '../train/'
    TRAIN_SUB_DIR = '../subsample/'
    TRAIN_GAN = '../../image_gan/'
    TEST_DIR = '../../test/'

    # use this for full dataset
    train_images_gan = [TRAIN_GAN + i for i in os.listdir(TRAIN_GAN)]
    test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
    train_images = train_images_gan
    
    train_images.sort(key=natural_keys)
    test_images.sort(key=natural_keys)

    # initialize the features matrix and labels list
    trainImage = []
    trainLabels = []
    testImage = []
    testLabels = []

    # loop over the input images
    for (i, imagePath) in enumerate(train_images):
        # extract the class label
        # get the labels from the name of the images by extract the string before "_"
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # read and resize image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        trainImage.append(img)
        trainLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
      # loop over the input images
    for (i, imagePath) in enumerate(test_images):
        # extract the class label
        # our images were named as labels.image_number.format
        # get the labels from the name of the images by extract the string before "."
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # extract CNN features in the image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        testImage.append(img)
        testLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


    trainImage = np.array(trainImage,dtype = float32)
    trainLabels = np.array(trainLabels)
    testImage = np.array(testImage,dtype = float32)
    testLabels = np.array(testLabels)
    print (trainImage.shape)
    
    trainImage = trainImage.astype(np.float32) / 255.0
    testImage = testImage.astype(np.float32) / 255.0
    
    le = preprocessing.LabelEncoder()
    le.fit(trainLabels)
    list(le.classes_)
    trainLabels = le.transform(trainLabels) 
    testLabels = le.transform(testLabels) 
    
    return trainImage, trainLabels, testImage, testLabels

In [3]:
trainImage, trainLabels, testImage, testLabels = load_flower_data()

nb_classes = 4

# Convert class vectors to binary class matrices.
trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
print (trainLabels)
testLabels = keras.utils.to_categorical(testLabels, nb_classes)
print (testLabels)
print (testLabels.shape)

np.save('../trainImage.npy', trainImage)
np.save('../trainLabels.npy', trainLabels)
np.save('../testImage.npy', testImage)
np.save('../testLabels.npy', testLabels)

print("[INFO] trainImage matrix: {:.2f}MB".format(
    (trainImage.nbytes) / (1024 * 1000.0)))
print("[INFO] trainLabels matrix: {:.4f}MB".format(
    (trainLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage matrix: {:.2f}MB".format(
    (testImage.nbytes) / (1024 * 1000.0)))
print("[INFO] testLabels matrix: {:.4f}MB".format(
    (testLabels.nbytes) / (1024 * 1000.0)))


[INFO] handling images...
[INFO] processed 1000/6000
[INFO] processed 2000/6000
[INFO] processed 3000/6000
[INFO] processed 4000/6000
[INFO] processed 5000/6000
[INFO] processed 6000/6000
[INFO] processed 308/308
(6000, 128, 128, 3)
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
(308, 4)
[INFO] trainImage matrix: 1152.00MB
[INFO] trainLabels matrix: 0.1875MB
[INFO] testImage matrix: 59.14MB
[INFO] testLabels matrix: 0.0096MB


In [4]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 4
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]    
        # number of dense filters to use  
        self.nb_Dense = [128] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # train   
        self.global_step = tf.Variable(0, trainable=False)
        self.initial_lr = 0.001 #初始学习率
        self.decay_steps = 10
        self.decay_rate = 0.9
        self.lr = tf.train.exponential_decay( self.initial_lr,
                                     global_step=self.global_step,
                                     decay_steps=self.decay_steps,
                                     decay_rate=self.decay_rate)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300

        # name for checkpoint
        self.model_name = 'CNN_C%d_D%d_Kernel(%d,%d)_%d_lrdecay0.0001' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))    
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y= np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y= np.load('../testLabels.npy')
        
        print ("Training:",self.train_x.shape[0])
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 

            #输入,卷积核为3*3 输出维度为32
            net = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1'               # 命名用于获取变量
                                    )
            print("CNN:",net.get_shape())

            if self.batch_normalization_control:
                net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            net = tf.layers.max_pooling2d(inputs = net,
                                          pool_size = self.pool_size,
                                          strides = (2, 2),
                                          padding='same',
                                          name='pool_conv_' + str(1)
                                         )
            print("CNN:",net.get_shape())
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )

                print("CNN:",net.get_shape())

            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
            
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'_train_hist.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status            
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, 
                                                                                    self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                rand_index = np.random.choice(len(self.train_x), size=batch_size)
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                   self.y: batch_y_tes,
                                                                   self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
            
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
                
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append(test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '_train_hist.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '_train_hist.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
    
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                               self.y: batch_y_tes,
                                                               self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")
        
    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                               self.y: batch_y_tes,
                                                               self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')
                          
        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [5]:
gan_type = 'CNN'
dataset = '4_Flowers'
epoch = 80
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
    #CNN.test(epoch)
        
sess.close()
        

# lrdecay
# Avg per epoch ptime: 11.07, total 80 epochs ptime: 949.30
#  [*] Training finished!
#  [*] Best Epoch:  47 , Accuracy:  0.766666571299235
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_C5_D1_Kernel(3,3)_128_lrdecay0.0001/CNN_C5_D1_Kernel(3,3)_128_lrdecay0.0001-47
#  [*] Finished testing Best Epoch: 47 , accuracy:  0.7666666507720947 

Training: 6000
CNN:x (100, 128, 128, 3)
CNN: (100, 128, 128, 32)
CNN: (100, 64, 64, 32)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 4)
------------------------
CNN:x (100, 128, 128, 3)
CNN: (100, 128, 128, 32)
CNN: (100, 64, 64, 32)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 4)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/batch_normalization/beta:0 (float32_ref 32) [32, bytes: 128]
cnn/batch_normalization/gamma:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_2/kernel:0 (float32_ref 3x3x32x64) [184

Epoch: [ 2] [  44/  60] time: 23.2912, loss: 0.24735904
Epoch: [ 2] [  45/  60] time: 23.4593, loss: 0.34132999
Epoch: [ 2] [  46/  60] time: 23.6324, loss: 0.27034709
Epoch: [ 2] [  47/  60] time: 23.7973, loss: 0.22004920
Epoch: [ 2] [  48/  60] time: 23.9675, loss: 0.21119279
Epoch: [ 2] [  49/  60] time: 24.1424, loss: 0.24342096
Epoch: [ 2] [  50/  60] time: 24.3183, loss: 0.22622752
Epoch: [ 2] [  51/  60] time: 24.4786, loss: 0.17698401
Epoch: [ 2] [  52/  60] time: 24.6584, loss: 0.21381320
Epoch: [ 2] [  53/  60] time: 24.8262, loss: 0.36518228
Epoch: [ 2] [  54/  60] time: 24.9925, loss: 0.15915675
Epoch: [ 2] [  55/  60] time: 25.1603, loss: 0.16435269
Epoch: [ 2] [  56/  60] time: 25.3331, loss: 0.28106138
Epoch: [ 2] [  57/  60] time: 25.5123, loss: 0.15491129
Epoch: [ 2] [  58/  60] time: 25.6871, loss: 0.25721556
Epoch: [ 2] [  59/  60] time: 25.8451, loss: 0.16969517
[2/80] - ptime: 11.0936 loss: 0.25680649 acc: 0.26333 lr: 0.00097915
Epoch: [ 3] [   0/  60] time: 28.05

Epoch: [ 5] [   8/  60] time: 53.1418, loss: 0.10105290
Epoch: [ 5] [   9/  60] time: 53.2994, loss: 0.11192763
Epoch: [ 5] [  10/  60] time: 53.4571, loss: 0.08088166
Epoch: [ 5] [  11/  60] time: 53.6099, loss: 0.09817481
Epoch: [ 5] [  12/  60] time: 53.7739, loss: 0.12062187
Epoch: [ 5] [  13/  60] time: 53.9275, loss: 0.11848396
Epoch: [ 5] [  14/  60] time: 54.0926, loss: 0.20775872
Epoch: [ 5] [  15/  60] time: 54.2476, loss: 0.11768459
Epoch: [ 5] [  16/  60] time: 54.4043, loss: 0.17965446
Epoch: [ 5] [  17/  60] time: 54.5639, loss: 0.11440388
Epoch: [ 5] [  18/  60] time: 54.7391, loss: 0.12960257
Epoch: [ 5] [  19/  60] time: 54.8930, loss: 0.06899586
Epoch: [ 5] [  20/  60] time: 55.0611, loss: 0.14951712
Epoch: [ 5] [  21/  60] time: 55.2166, loss: 0.13085893
Epoch: [ 5] [  22/  60] time: 55.3812, loss: 0.19532709
Epoch: [ 5] [  23/  60] time: 55.5367, loss: 0.18036966
Epoch: [ 5] [  24/  60] time: 55.6975, loss: 0.08643995
Epoch: [ 5] [  25/  60] time: 55.8595, loss: 0.1

Epoch: [ 7] [  32/  60] time: 81.7105, loss: 0.15108907
Epoch: [ 7] [  33/  60] time: 81.8683, loss: 0.10978971
Epoch: [ 7] [  34/  60] time: 82.0276, loss: 0.09376565
Epoch: [ 7] [  35/  60] time: 82.1810, loss: 0.10659982
Epoch: [ 7] [  36/  60] time: 82.3480, loss: 0.10925054
Epoch: [ 7] [  37/  60] time: 82.5065, loss: 0.13629645
Epoch: [ 7] [  38/  60] time: 82.6686, loss: 0.07214858
Epoch: [ 7] [  39/  60] time: 82.8240, loss: 0.12123884
Epoch: [ 7] [  40/  60] time: 82.9880, loss: 0.16265260
Epoch: [ 7] [  41/  60] time: 83.1476, loss: 0.11819346
Epoch: [ 7] [  42/  60] time: 83.3145, loss: 0.11828925
Epoch: [ 7] [  43/  60] time: 83.4691, loss: 0.09213077
Epoch: [ 7] [  44/  60] time: 83.6355, loss: 0.10989610
Epoch: [ 7] [  45/  60] time: 83.7887, loss: 0.09993499
Epoch: [ 7] [  46/  60] time: 83.9528, loss: 0.16492637
Epoch: [ 7] [  47/  60] time: 84.1123, loss: 0.12770517
Epoch: [ 7] [  48/  60] time: 84.2819, loss: 0.16133352
Epoch: [ 7] [  49/  60] time: 84.4350, loss: 0.1

Epoch: [ 9] [  56/  60] time: 109.2958, loss: 0.08574638
Epoch: [ 9] [  57/  60] time: 109.4508, loss: 0.01756824
Epoch: [ 9] [  58/  60] time: 109.6123, loss: 0.12406386
Epoch: [ 9] [  59/  60] time: 109.7668, loss: 0.05197138
[9/80] - ptime: 10.9502 loss: 0.08306673 acc: 0.69333 lr: 0.00090953
Epoch: [10] [   0/  60] time: 112.0591, loss: 0.03774494
Epoch: [10] [   1/  60] time: 112.2249, loss: 0.07395053
Epoch: [10] [   2/  60] time: 112.3942, loss: 0.05227523
Epoch: [10] [   3/  60] time: 112.5502, loss: 0.08461619
Epoch: [10] [   4/  60] time: 112.7150, loss: 0.06047730
Epoch: [10] [   5/  60] time: 112.8734, loss: 0.06281602
Epoch: [10] [   6/  60] time: 113.0338, loss: 0.07114916
Epoch: [10] [   7/  60] time: 113.1943, loss: 0.09827320
Epoch: [10] [   8/  60] time: 113.3549, loss: 0.05386765
Epoch: [10] [   9/  60] time: 113.5130, loss: 0.12276579
Epoch: [10] [  10/  60] time: 113.6761, loss: 0.02015604
Epoch: [10] [  11/  60] time: 113.8317, loss: 0.07923395
Epoch: [10] [  12/ 

...

Epoch: [71] [  28/  60] time: 837.4777, loss: 0.00006007
Epoch: [71] [  29/  60] time: 837.6332, loss: 0.00086597
Epoch: [71] [  30/  60] time: 837.7999, loss: 0.00015855
Epoch: [71] [  31/  60] time: 837.9549, loss: 0.00015278
Epoch: [71] [  32/  60] time: 838.1167, loss: 0.00014585
Epoch: [71] [  33/  60] time: 838.2800, loss: 0.00006729
Epoch: [71] [  34/  60] time: 838.4421, loss: 0.00004669
Epoch: [71] [  35/  60] time: 838.6037, loss: 0.00085164
Epoch: [71] [  36/  60] time: 838.7666, loss: 0.00012289
Epoch: [71] [  37/  60] time: 838.9209, loss: 0.00005267
Epoch: [71] [  38/  60] time: 839.0853, loss: 0.00147419
Epoch: [71] [  39/  60] time: 839.2391, loss: 0.00058848
Epoch: [71] [  40/  60] time: 839.4074, loss: 0.00029478
Epoch: [71] [  41/  60] time: 839.5665, loss: 0.00001665
Epoch: [71] [  42/  60] time: 839.7336, loss: 0.00012580
Epoch: [71] [  43/  60] time: 839.8913, loss: 0.00020688
Epoch: [71] [  44/  60] time: 840.0541, loss: 0.00007618
Epoch: [71] [  45/  60] time: 8

Epoch: [73] [  50/  60] time: 864.3556, loss: 0.00005935
Epoch: [73] [  51/  60] time: 864.5122, loss: 0.00001808
Epoch: [73] [  52/  60] time: 864.6773, loss: 0.00016857
Epoch: [73] [  53/  60] time: 864.8280, loss: 0.00006931
Epoch: [73] [  54/  60] time: 864.9891, loss: 0.00000736
Epoch: [73] [  55/  60] time: 865.1455, loss: 0.00009008
Epoch: [73] [  56/  60] time: 865.3104, loss: 0.00053983
Epoch: [73] [  57/  60] time: 865.4681, loss: 0.00085942
Epoch: [73] [  58/  60] time: 865.6361, loss: 0.00014871
Epoch: [73] [  59/  60] time: 865.7880, loss: 0.00022662
[73/80] - ptime: 10.8515 loss: 0.00019124 acc: 0.74667 lr: 0.00046342
Epoch: [74] [   0/  60] time: 868.5972, loss: 0.00021450
Epoch: [74] [   1/  60] time: 868.7538, loss: 0.00009335
Epoch: [74] [   2/  60] time: 868.9152, loss: 0.00001982
Epoch: [74] [   3/  60] time: 869.0757, loss: 0.00003286
Epoch: [74] [   4/  60] time: 869.2469, loss: 0.00011158
Epoch: [74] [   5/  60] time: 869.4029, loss: 0.00006110
Epoch: [74] [   6/

Epoch: [76] [  12/  60] time: 894.0546, loss: 0.00004421
Epoch: [76] [  13/  60] time: 894.2110, loss: 0.00106976
Epoch: [76] [  14/  60] time: 894.3762, loss: 0.00001816
Epoch: [76] [  15/  60] time: 894.5322, loss: 0.00212200
Epoch: [76] [  16/  60] time: 894.6965, loss: 0.00000772
Epoch: [76] [  17/  60] time: 894.8521, loss: 0.00007684
Epoch: [76] [  18/  60] time: 895.0203, loss: 0.00002309
Epoch: [76] [  19/  60] time: 895.1779, loss: 0.00002597
Epoch: [76] [  20/  60] time: 895.3459, loss: 0.00001598
Epoch: [76] [  21/  60] time: 895.5011, loss: 0.00003376
Epoch: [76] [  22/  60] time: 895.6631, loss: 0.00013044
Epoch: [76] [  23/  60] time: 895.8206, loss: 0.00010024
Epoch: [76] [  24/  60] time: 896.0013, loss: 0.00008362
Epoch: [76] [  25/  60] time: 896.1586, loss: 0.00000916
Epoch: [76] [  26/  60] time: 896.3253, loss: 0.00000992
Epoch: [76] [  27/  60] time: 896.4833, loss: 0.00124556
Epoch: [76] [  28/  60] time: 896.6484, loss: 0.00003694
Epoch: [76] [  29/  60] time: 8

Epoch: [78] [  34/  60] time: 921.0584, loss: 0.00005839
Epoch: [78] [  35/  60] time: 921.2162, loss: 0.00013456
Epoch: [78] [  36/  60] time: 921.3822, loss: 0.00001096
Epoch: [78] [  37/  60] time: 921.5380, loss: 0.00001111
Epoch: [78] [  38/  60] time: 921.7022, loss: 0.00004489
Epoch: [78] [  39/  60] time: 921.8664, loss: 0.00000675
Epoch: [78] [  40/  60] time: 922.0294, loss: 0.00001630
Epoch: [78] [  41/  60] time: 922.1877, loss: 0.00012013
Epoch: [78] [  42/  60] time: 922.3518, loss: 0.00000575
Epoch: [78] [  43/  60] time: 922.5035, loss: 0.00002030
Epoch: [78] [  44/  60] time: 922.6660, loss: 0.00009706
Epoch: [78] [  45/  60] time: 922.8228, loss: 0.00013784
Epoch: [78] [  46/  60] time: 922.9955, loss: 0.00007118
Epoch: [78] [  47/  60] time: 923.1550, loss: 0.00000864
Epoch: [78] [  48/  60] time: 923.3169, loss: 0.00196053
Epoch: [78] [  49/  60] time: 923.4724, loss: 0.00007940
Epoch: [78] [  50/  60] time: 923.6334, loss: 0.00259573
Epoch: [78] [  51/  60] time: 9

Epoch: [80] [  56/  60] time: 947.8986, loss: 0.00010474
Epoch: [80] [  57/  60] time: 948.0576, loss: 0.00010121
Epoch: [80] [  58/  60] time: 948.2162, loss: 0.00007195
Epoch: [80] [  59/  60] time: 948.3727, loss: 0.00073577
[80/80] - ptime: 10.7718 loss: 0.00103824 acc: 0.76667 lr: 0.00043047
Avg per epoch ptime: 11.07, total 80 epochs ptime: 949.30
 [*] Training finished!
 [*] Best Epoch:  47 , Accuracy:  0.766666571299235
INFO:tensorflow:Restoring parameters from checkpoint/CNN_C5_D1_Kernel(3,3)_128_lrdecay0.0001/CNN_C5_D1_Kernel(3,3)_128_lrdecay0.0001-47
 [*] Finished testing Best Epoch: 47 , accuracy:  0.7666666507720947 !
